In [1]:
import sys
import os
sys.path.append('../dags')

from database import create_database_engine


# 환경 변수 설정
os.environ["POSTGRES_USER"] = "airflow"
os.environ["POSTGRES_PASSWORD"] = "airflow"
os.environ["POSTGRES_DB"] = "events"
os.environ["POSTGRES_PORT"] = "5433"

engine = create_database_engine(host="127.0.0.1")
print("Successfully connected to the database")

Successfully connected to the database


In [2]:
import pandas as pd
from datetime import datetime

# 변수 설정
label_table = "kind"
return_table = "abnormal_return_kind_log"
volume_table = "abnormal_return_kind"   # volume은 이 테이블에서 select
label_col = "label"

# abnormal return 컬럼
abn_return_cols = [
    "abn_ret_minus_9m", "abn_ret_minus_8m", "abn_ret_minus_7m", "abn_ret_minus_6m",
    "abn_ret_minus_5m", "abn_ret_minus_4m", "abn_ret_minus_3m", "abn_ret_minus_2m", "abn_ret_minus_1m", "abn_ret_0m",
    "abn_ret_1m", "abn_ret_2m", "abn_ret_3m", "abn_ret_4m", "abn_ret_5m", "abn_ret_6m",
    "abn_ret_7m", "abn_ret_8m", "abn_ret_9m", "abn_ret_10m"
]

# volume 컬럼
volume_cols = [
    "volume_minus_10m", "volume_minus_9m", "volume_minus_8m", "volume_minus_7m", "volume_minus_6m",
    "volume_minus_5m", "volume_minus_4m", "volume_minus_3m", "volume_minus_2m", "volume_minus_1m",
    "volume_1m", "volume_2m", "volume_3m", "volume_4m", "volume_5m", "volume_6m",
    "volume_7m", "volume_8m", "volume_9m", "volume_10m"
]

# SQL 컬럼 문자열 (event_ts는 return_table에서, abn_*은 return_table, volume_*은 volume_table, label은 kind에서)
sql_columns = (
    [f"ar.event_ts"] +
    [f"ar.{col}" for col in abn_return_cols] +
    [f"vol.{col}" for col in volume_cols] +
    [f"k.{label_col}"]
)
sql_columns_str = ",\n    ".join(sql_columns)

query = f"""
SELECT {sql_columns_str}
FROM {return_table} ar
JOIN {volume_table} vol ON ar.event_id = vol.event_id
JOIN "{label_table}" k ON ar.event_id = k.id
ORDER BY ar.event_ts ASC
"""

df_total = pd.read_sql(query, engine)
df_total["period_dummy"] = (
    pd.to_datetime(df_total["event_ts"], utc=True) > pd.Timestamp("2022-06-30", tz="UTC")
).astype(int)
df_total

DETAIL:  The database was created using collation version 2.36, but the operating system provides version 2.41.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE events REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


,event_ts,abn_ret_minus_9m,abn_ret_minus_8m,abn_ret_minus_7m,abn_ret_minus_6m,abn_ret_minus_5m,abn_ret_minus_4m,abn_ret_minus_3m,abn_ret_minus_2m,abn_ret_minus_1m,...,volume_3m,volume_4m,volume_5m,volume_6m,volume_7m,volume_8m,volume_9m,volume_10m,label,period_dummy
0,2021-01-04 10:08:00+00:00,0.000000,0.009472,-0.001189,-0.006321,-0.004620,0.003331,-0.004050,0.007088,-0.002013,...,738.0,3081.0,6644.0,3178.0,1059.0,1373.0,1244.0,1457.0,0,0
1,2021-01-04 10:08:00+00:00,0.000000,0.009472,-0.001189,-0.006321,-0.004620,0.003331,-0.004050,0.007088,-0.002013,...,738.0,3081.0,6644.0,3178.0,1059.0,1373.0,1244.0,1457.0,0,0
2,2021-01-04 10:08:00+00:00,NaN,NaN,-0.001945,0.001952,-0.008174,-0.002652,-0.001960,0.007570,-0.005890,...,11510.0,2956.0,3751.0,1782.0,4513.0,337.0,34.0,1704.0,1,0
3,2021-01-04 10:14:00+00:00,-0.001960,0.003144,0.000105,0.001302,-0.004985,0.002281,0.000653,0.002719,0.003376,...,167457.0,74079.0,15460.0,17484.0,11350.0,12145.0,38281.0,10831.0,1,0
4,2021-01-04 10:16:00+00:00,0.002785,-0.002659,-0.000252,0.000252,0.000000,-0.000775,-0.001412,-0.003190,0.001265,...,1588.0,1805.0,5776.0,6845.0,880.0,5369.0,3481.0,840.0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8651,2023-12-28 14:46:00+00:00,-0.001304,0.000944,0.000000,0.000000,0.000360,0.000000,-0.001304,-0.000360,0.000000,...,8934.0,19058.0,27278.0,14587.0,7554.0,6315.0,5149.0,2479.0,1,1
8652,2023-12-28 14:53:00+00:00,0.007121,0.000417,-0.000555,-0.006324,0.000278,-0.000139,0.000000,-0.000139,-0.000416,...,82.0,2001.0,210.0,32.0,237.0,70.0,70.0,70.0,1,1
8653,2023-12-28 14:56:00+00:00,0.001974,0.001776,-0.000139,0.000000,-0.000139,-0.000416,-0.000693,0.000000,0.000277,...,154.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,0,1
8654,2023-12-28 14:56:00+00:00,0.001974,0.001776,-0.000139,0.000000,-0.000139,-0.000416,-0.000693,0.000000,0.000277,...,154.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,0,1


In [3]:
for m in range(1, 11):
    plus_cols = [
        f"abn_ret_minus_{abs(i)}m" if i < 0 else f"abn_ret_{i}m"
        for i in range(-m+1, m + 1)
    ]
    df_total[f"CAR_{m}m"] = df_total[plus_cols].sum(axis=1)

In [4]:
df_total

,event_ts,abn_ret_minus_9m,abn_ret_minus_8m,abn_ret_minus_7m,abn_ret_minus_6m,abn_ret_minus_5m,abn_ret_minus_4m,abn_ret_minus_3m,abn_ret_minus_2m,abn_ret_minus_1m,...,CAR_1m,CAR_2m,CAR_3m,CAR_4m,CAR_5m,CAR_6m,CAR_7m,CAR_8m,CAR_9m,CAR_10m
0,2021-01-04 10:08:00+00:00,0.000000,0.009472,-0.001189,-0.006321,-0.004620,0.003331,-0.004050,0.007088,-0.002013,...,0.027264,0.014314,0.023078,0.022705,0.022186,0.017195,0.010830,0.005286,0.018039,0.016961
1,2021-01-04 10:08:00+00:00,0.000000,0.009472,-0.001189,-0.006321,-0.004620,0.003331,-0.004050,0.007088,-0.002013,...,0.027264,0.014314,0.023078,0.022705,0.022186,0.017195,0.010830,0.005286,0.018039,0.016961
2,2021-01-04 10:08:00+00:00,NaN,NaN,-0.001945,0.001952,-0.008174,-0.002652,-0.001960,0.007570,-0.005890,...,0.024983,0.016555,0.000327,0.000001,-0.000359,0.001975,-0.000003,0.000671,0.003952,-0.004914
3,2021-01-04 10:14:00+00:00,-0.001960,0.003144,0.000105,0.001302,-0.004985,0.002281,0.000653,0.002719,0.003376,...,0.081766,0.082847,0.060447,0.046534,0.052603,0.048936,0.048975,0.043475,0.041041,0.037109
4,2021-01-04 10:16:00+00:00,0.002785,-0.002659,-0.000252,0.000252,0.000000,-0.000775,-0.001412,-0.003190,0.001265,...,0.006372,0.007764,0.002406,0.000360,-0.001681,-0.004744,-0.002693,-0.004713,-0.009930,-0.001911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8651,2023-12-28 14:46:00+00:00,-0.001304,0.000944,0.000000,0.000000,0.000360,0.000000,-0.001304,-0.000360,0.000000,...,0.052426,0.106493,0.105773,0.146132,0.085906,0.091794,0.094865,0.088712,0.094455,0.089213
8652,2023-12-28 14:53:00+00:00,0.007121,0.000417,-0.000555,-0.006324,0.000278,-0.000139,0.000000,-0.000139,-0.000416,...,0.007160,-0.006304,0.002384,0.002849,0.001967,0.002245,-0.003940,-0.004079,-0.003662,0.003182
8653,2023-12-28 14:56:00+00:00,0.001974,0.001776,-0.000139,0.000000,-0.000139,-0.000416,-0.000693,0.000000,0.000277,...,-0.004917,-0.000277,-0.000277,-0.000831,-0.000831,-0.000970,-0.001247,-0.001386,0.000113,0.002087
8654,2023-12-28 14:56:00+00:00,0.001974,0.001776,-0.000139,0.000000,-0.000139,-0.000416,-0.000693,0.000000,0.000277,...,-0.004917,-0.000277,-0.000277,-0.000831,-0.000831,-0.000970,-0.001247,-0.001386,0.000113,0.002087


In [5]:
for m in range(1, 11):
    minus_cols = [f"volume_minus_{x}m" for x in range(m, 0, -1)]
    df_total[f"cum_volume_minus_{m}m"] = df_total[minus_cols].sum(axis=1)

# 누적 거래량 (cum_volume_1m ~ cum_volume_10m)
for m in range(1, 11):
    plus_cols = [f"volume_{x}m" for x in range(1, m + 1)]
    df_total[f"cum_volume_{m}m"] = df_total[plus_cols].sum(axis=1)
    

# volume으로 시작하는 컬럼 모두 drop
volume_cols_to_drop = [col for col in df_total.columns if col.startswith("volume")]
df_total = df_total.drop(columns=volume_cols_to_drop)


In [6]:
df_total.filter(regex="^cum_").head()

,cum_volume_minus_1m,cum_volume_minus_2m,cum_volume_minus_3m,cum_volume_minus_4m,cum_volume_minus_5m,cum_volume_minus_6m,cum_volume_minus_7m,cum_volume_minus_8m,cum_volume_minus_9m,cum_volume_minus_10m,cum_volume_1m,cum_volume_2m,cum_volume_3m,cum_volume_4m,cum_volume_5m,cum_volume_6m,cum_volume_7m,cum_volume_8m,cum_volume_9m,cum_volume_10m
0,175.0,2129.0,4899.0,5995.0,12027.0,14258.0,20333.0,25327.0,25347.0,25367.0,16549.0,27978.0,28716.0,31797.0,38441.0,41619.0,42678.0,44051.0,45295.0,46752.0
1,175.0,2129.0,4899.0,5995.0,12027.0,14258.0,20333.0,25327.0,25347.0,25367.0,16549.0,27978.0,28716.0,31797.0,38441.0,41619.0,42678.0,44051.0,45295.0,46752.0
2,678.0,3414.0,4754.0,9270.0,11428.0,12870.0,13198.0,18159.0,18159.0,18159.0,19277.0,31427.0,42937.0,45893.0,49644.0,51426.0,55939.0,56276.0,56310.0,58014.0
3,549.0,557.0,658.0,811.0,1540.0,2617.0,3382.0,7552.0,7629.0,14141.0,172251.0,344502.0,511959.0,586038.0,601498.0,618982.0,630332.0,642477.0,680758.0,691589.0
4,18708.0,31931.0,38483.0,41085.0,45101.0,62675.0,71568.0,78630.0,83825.0,98085.0,6055.0,11175.0,12763.0,14568.0,20344.0,27189.0,28069.0,33438.0,36919.0,37759.0


In [7]:
import numpy as np

# delta_cum_volume_{m}m = (cum_volume_{m}m - cum_volume_minus_{m}m) / cum_volume_minus_{m}
for m in range(1, 11):
    denom = 1 + df_total[f"cum_volume_minus_{m}m"]
    num = 1 + df_total[f"cum_volume_{m}m"]
    df_total[f"delta_cum_volume_{m}m"] = np.log(num / denom)

# min-max scaling for each delta_cum_volume_{m}m to [-1, 1]
for m in range(1, 11):
    col = f"delta_cum_volume_{m}m"
    x = df_total[col].values
    x_min = np.nanmin(x)
    x_max = np.nanmax(x)
    df_total[col] = 2 * (x - x_min) / (x_max - x_min) - 1

# 누적 거래량으로 시작하는 컬럼 모두 drop
cum_cols_to_drop = [col for col in df_total.columns if col.startswith("cum_volume_")]
df_total = df_total.drop(columns=cum_cols_to_drop)
df_total.filter(regex="^delta_cum_").head()


,delta_cum_volume_1m,delta_cum_volume_2m,delta_cum_volume_3m,delta_cum_volume_4m,delta_cum_volume_5m,delta_cum_volume_6m,delta_cum_volume_7m,delta_cum_volume_8m,delta_cum_volume_9m,delta_cum_volume_10m
0,0.333486,0.161402,0.088946,0.085629,0.044568,0.036476,0.011251,-0.008535,-0.009986,-0.007279
1,0.333486,0.161402,0.088946,0.085629,0.044568,0.036476,0.011251,-0.008535,-0.009986,-0.007279
2,0.232936,0.132261,0.123457,0.080221,0.068455,0.060749,0.065219,0.035202,0.031363,0.033821
3,0.434504,0.476734,0.479074,0.471461,0.418649,0.376194,0.355772,0.286021,0.283405,0.237671
4,-0.142715,-0.135503,-0.140313,-0.126794,-0.107865,-0.110893,-0.117568,-0.115191,-0.115059,-0.124281


In [8]:
df_total.filter(regex="^delta_cum_").describe()


,delta_cum_volume_1m,delta_cum_volume_2m,delta_cum_volume_3m,delta_cum_volume_4m,delta_cum_volume_5m,delta_cum_volume_6m,delta_cum_volume_7m,delta_cum_volume_8m,delta_cum_volume_9m,delta_cum_volume_10m
count,8656.000000,8656.000000,8656.000000,8656.000000,8656.000000,8656.000000,8656.000000,8656.000000,8656.000000,8656.000000
mean,0.163739,0.136650,0.116684,0.110005,0.098784,0.089216,0.082213,0.070207,0.061515,0.057974
std,0.246935,0.217868,0.200190,0.188526,0.180285,0.173094,0.167554,0.161789,0.157080,0.153931
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,-0.021705,-0.017887,-0.022577,-0.018487,-0.022234,-0.024684,-0.025981,-0.033382,-0.038513,-0.038890
50%,0.144107,0.111970,0.093918,0.086244,0.074949,0.066358,0.061280,0.049647,0.041583,0.039072
75%,0.329797,0.273464,0.238136,0.221723,0.203932,0.190321,0.178890,0.163284,0.150244,0.144150
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
df_total[df_total["period_dummy"] == 1].head()

,event_ts,abn_ret_minus_9m,abn_ret_minus_8m,abn_ret_minus_7m,abn_ret_minus_6m,abn_ret_minus_5m,abn_ret_minus_4m,abn_ret_minus_3m,abn_ret_minus_2m,abn_ret_minus_1m,...,delta_cum_volume_1m,delta_cum_volume_2m,delta_cum_volume_3m,delta_cum_volume_4m,delta_cum_volume_5m,delta_cum_volume_6m,delta_cum_volume_7m,delta_cum_volume_8m,delta_cum_volume_9m,delta_cum_volume_10m
5072,2022-07-01 10:03:00+00:00,0.011478,0.008055,0.000474,-0.003784,-0.000001,0.004726,-0.004252,0.000000,0.001898,...,-0.197859,-0.124490,-0.068736,-0.113969,-0.110240,-0.064567,-0.066950,-0.054036,-0.055558,-0.070355
5073,2022-07-01 10:24:00+00:00,0.001432,0.000000,0.000000,0.001434,0.000479,0.000479,-0.004817,0.000000,-0.001435,...,0.239031,0.207561,0.179545,0.193374,0.191108,0.189815,0.149814,0.117653,0.101474,0.086810
5074,2022-07-01 10:26:00+00:00,-0.001436,0.001434,-0.000959,0.000479,-0.001919,0.000000,0.000005,0.000956,0.001916,...,0.136659,0.130844,0.152217,0.140227,0.135902,0.137003,0.139673,0.065761,0.060644,0.069101
5075,2022-07-01 10:43:00+00:00,-0.001049,-0.001140,-0.000238,0.000979,-0.000979,-0.000489,-0.000815,-0.000814,0.000000,...,-0.059102,-0.055807,-0.057294,-0.054096,-0.076047,-0.077524,-0.077126,-0.071888,-0.063153,-0.066351
5076,2022-07-01 11:03:00+00:00,-0.000936,-0.000450,0.000650,0.002360,0.000000,-0.000936,0.001749,-0.000976,-0.000773,...,-0.077906,-0.082003,0.024876,0.059036,0.049761,0.044802,0.023395,0.014264,0.005169,-0.006780


# 방향 정합성

### 📘 모형식 정의

$$
\text{logit}^{-1}(x) = \frac{1}{1 + e^{-x}}
$$

$$
Pr(hit_{i,t} = 1) = \text{logit}^{-1}(\alpha + \beta \cdot period\_dummy_i)
$$
---

### 📊 변수 설명


이벤트 *i*, 창 *t* 에 대해 Delta CAR, post CAR 2개의 지표를 사용

$$
t \in \{10,\,20,\,\dots,\,60\}
$$

$$
\Delta CAR_{i,t} = CAR_{post,i,t} - CAR_{pre,i,t}  
$$

$$
CAR_{post,i,t}
$$

$$
label\_sign_i =
\begin{cases}
+1, & \text{긍정 (1)} \\
0, & \text{중립 (0)} \\
-1, & \text{부정 (-1)} 
\end{cases}
$$



$$
hit_{i,t} =
\begin{cases}
1, & \text{if } \operatorname{sign}(\Delta CAR_{i,t}) = label\_sign_i \\
0, & \text{otherwise}
\end{cases}
$$

---

### 📑 가설 설정

$$
\begin{aligned}
H_0 &: \beta = 0 \quad &(\text{도입 전후 차이 없음}) \\
H_1 &: \beta > 0 \quad &(\text{도입 후 정합률 상승})
\end{aligned}
$$

---

### 📈 해석

- delta CAR: β > 0 이고 **통계적으로 유의**하다면  → “텔레그램 서비스 출시 이후 과잉반응 감소 / 즉시 반영 후 빠른 수렴 → 정보의 선반영 및 사전 확산이 강화 → 정보 효율성 향상”을 의미한다고 볼 수 있다.
- Post CAR: β > 0 이고 **통계적으로 유의**하다면  → “텔레그램 서비스 출시 이후 이벤트 발표 이후의 정합률(정확도) 증가 → 시장 반응의 즉시 반영 강화 → 정보 효율성 향상”을 의미한다고 해석할 수 있다.


“중립 이벤트를 포함한 모델에서는 텔레그램 도입 이후 정합률이 유의하게 상승하였으나,
중립을 제외한 경우에는 그 효과가 약화되었다. 이는 정보 효율성 개선이
주로 가격 반응의 ‘적정성(overshooting 감소)’ 혹은 ‘비과잉 반응’ 영역에서 나타났음을 시사한다.
다시 말해, 시장이 새로운 정보를 과도하게 반응하지 않고 신속히 흡수하게 된 결과로 볼 수 있다.”

또는 요약형으로:

“효율성 개선은 방향성 예측력이 아니라, 불필요한 반응을 줄이는 형태로 나타났다.”

## 중립 포함

In [10]:
import sys
import os
sys.path.append('.')

from hypothesis_test_volume import (
    print_sample_summary_with_neutral,
    logistic_hit_delta_with_neutral_and_vol_log,
    run_logistic_table
)

# === 전체 파이프라인 (중립 포함, 중립은 ΔCAR/POSTCAR 절대값 기준) ===

df = df_total.copy()
df["label_sign"] = df["label"]

df_full = print_sample_summary_with_neutral(df, "label_sign")
# 창 목록
windows = list(range(1, 11))

# ΔCAR + 거래량 공변량 (중립 포함)
results_delta_w_neutral_vol = run_logistic_table(
    logistic_hit_delta_with_neutral_and_vol_log,
    df_full,
    windows,
    label="ΔCAR + volume (neutral incl.)",
    covar_prefix="delta_cum_volume_",  # 기본값과 동일
    add_interaction=False,             # 상호작용 보려면 True
    robust_se=True
)



=== Sample summary (including neutral) ===
Total events:        8,656
Neutrals:            2,980
  - Positive (1):    4,906
  - Negative (-1):   770


--- ΔCAR + volume (neutral incl.) 기준 결과 ---
 t beta_star    std  t_stat  p_value  odds_ratio  p_before  p_after  diff_pp  n_obs     R2  adj_R2  neutral_epsilon gamma_vol  std_vol    p_vol
 1   0.1115* 0.0469   2.380 0.017299       1.118     0.665    0.689     2.44   8656 0.0005 -0.0000         0.005135   -0.0256   0.0942 0.785569
 2   0.1089* 0.0473   2.302 0.021349       1.115     0.675    0.699     2.34   8656 0.0011  0.0005         0.005668 -0.2786**   0.1068 0.009073
 3 0.1563*** 0.0470   3.329 0.000872       1.169     0.660    0.694     3.42   8656 0.0010  0.0005         0.006115   -0.0641   0.1157 0.579407
 4  0.1395** 0.0468   2.983 0.002859       1.150     0.657    0.687     3.07   8656 0.0008  0.0003         0.005996   -0.0467   0.1217 0.701212
 5 0.1728*** 0.0467   3.701 0.000214       1.189     0.650    0.688     3.82   8656 0

## 중립 제거

In [11]:
from hypothesis_test_volume import (
    print_sample_summary,
    logistic_hit_delta_with_vol_log,
    run_logistic_table
)

# === 전체 파이프라인 ===

df = df_total.copy()
df["label_sign"] = df["label"]
df_nn = print_sample_summary(df, "label_sign")
windows = list(range(1, 11))

# ΔCAR + volume (중립 제거)
results_delta_vol = run_logistic_table(
    logistic_hit_delta_with_vol_log,
    df_nn, windows,
    label="ΔCAR + volume (neutral removed)",
    covar_prefix="delta_cum_volume_",  # 이미 만드신 컬럼 접두사
    add_interaction=False,             # 필요 시 True
    robust_se=True
)



=== Sample summary (neutral removed) ===
Total events:        8,656
Removed neutrals:    2,980
Used (non-neutral):  5,676
  - Positive (1):    4,906
  - Negative (-1):   770


--- ΔCAR + volume (neutral removed) 기준 결과 ---
 t beta_star    std  t_stat  p_value  odds_ratio  p_before  p_after  diff_pp  n_obs     R2  adj_R2 gamma_vol  std_vol  p_vol
 1   -0.0715 0.0653  -1.095 0.273624       0.931     0.776    0.764    -1.19   5676 0.0525  0.0515 2.4365***   0.1465    0.0
 2   -0.0897 0.0654  -1.373 0.169749       0.914     0.785    0.770    -1.50   5676 0.0332  0.0322 2.1678***   0.1690    0.0
 3   -0.0389 0.0641  -0.607 0.543949       0.962     0.766    0.759    -0.68   5676 0.0363  0.0353 2.4547***   0.1851    0.0
 4   -0.0541 0.0630  -0.858 0.390838       0.947     0.756    0.747    -0.97   5676 0.0328  0.0319 2.4613***   0.1932    0.0
 5    0.0375 0.0626   0.600 0.548647       1.038     0.745    0.752     0.69   5676 0.0269  0.0260 2.2979***   0.1952    0.0
 6   -0.0003 0.0619  -0.004 